<a href="https://colab.research.google.com/github/Paulo-vitorCS/DeepLearning/blob/main/Projeto_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Projeto 13: Aumento de Imagens
###Rede Neural Convolucional MNIST

###Etapa 01: Importação das Bibliotecas

In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets  # Acessar os datasets instalados no pytorch
import torchvision.transforms as transforms  # Transformações e carregamento das imagens

###Etapa 02: Base de dados

In [2]:
torch.manual_seed(123)

In [3]:
# Transformador para a base de dados de treinamento
transform_train = transforms.Compose([
                                      transforms.RandomHorizontalFlip(),  # Girar a imagem na horizontal
                                      transforms.RandomAffine(degrees = 7,  # Graus
                                                              translate = (0, 0.07),  # Horizontal and vertical shifts
                                                              shear = 7,
                                                              scale = (1, 1.2)  # Zoom range escala da imagem (tamanho original, zoom)
                                                              ),
                                      transforms.ToTensor()
                                    ])

In [4]:
transform_test = transforms.ToTensor()  # Transformador para a base de dados de teste

In [6]:
train = datasets.MNIST(root = '.', train = True, download = True, transform = transform_train)
test = datasets.MNIST(root = '.', train = False, download = True, transform = transform_test)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [7]:
train_loader = torch.utils.data.DataLoader(train, batch_size = 128)
test_loader = torch.utils.data.DataLoader(test, batch_size = 128)

###Etapa 03: Construção do modelo

In [11]:
 class classificador(nn.Module):
     def __init__(self):
         super().__init__()
         self.conv = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = (3, 3))
         self.activation = nn.ReLU()
         self.pool = nn.MaxPool2d(kernel_size = (2, 2))
         self.flatten = nn.Flatten()
         self.linear = nn.Linear(in_features = 32*13*13, out_features = 128)
         self.output = nn.Linear(128, 10)
     
     def forward(self, X):
         X = self.pool(self.activation(self.conv(X)))
         X = self.flatten(X)
         X = self.activation(self.linear(X))
         X = self.output(X)
                
         return X

In [12]:
net = classificador()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [14]:
device

device(type='cuda')

In [15]:
net.to(device)

classificador(
  (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (activation): ReLU()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=5408, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
)

###Etapa 04: Treinamento do Modelo

In [16]:
def training_loop(loader, epoch):
    running_loss = 0.
    running_accuracy = 0. 

    for i, data in enumerate(loader):
        inputs, labels = data 
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        ps = F.softmax(outputs)
        top_p, top_class = ps.topk(k = 1, dim = 1)
        equals = top_class == labels.view(*top_class.shape)

        accuracy = torch.mean(equals.type(torch.float))

        running_accuracy += accuracy

        # Imprimindo os dados referentes a este loop:
        print('\rÉpoca {:3d} - Loop {:3d} de {:3d}: perda {:03.2f} - precisão {:03.2f}'.format(epoch + 1,
                                                                                               i + 1,
                                                                                               len(loader),
                                                                                               loss,
                                                                                               accuracy), end = '\r')
    
    # Imprimindo os dados referentes a esta época:
    print('\rÉpoca {:3d} FINALIZADA: perda {:.5f} - precisão {:.5f}'.format(epoch + 1,
                                                                              running_loss/len(loader),
                                                                              running_accuracy/len(loader)))

In [17]:
for epoch in range(5):
    print('Treinando...')
    training_loop(train_loader, epoch)

    net.eval()
    print('Validando...')
    training_loop(test_loader, epoch)

    net.train()

Treinando...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Época   1 FINALIZADA: perda 0.61234 - precisão 0.80511
Validando...
Época   1 FINALIZADA: perda 0.18265 - precisão 0.94432
Treinando...
Época   2 FINALIZADA: perda 0.26333 - precisão 0.91785
Validando...
Época   2 FINALIZADA: perda 0.10852 - precisão 0.96608
Treinando...
Época   3 FINALIZADA: perda 0.19534 - precisão 0.94007
Validando...
Época   3 FINALIZADA: perda 0.08011 - precisão 0.97498
Treinando...
Época   4 FINALIZADA: perda 0.16121 - precisão 0.95018
Validando...
Época   4 FINALIZADA: perda 0.05556 - precisão 0.98250
Treinando...
Época   5 FINALIZADA: perda 0.13973 - precisão 0.95663
Validando...
Época   5 FINALIZADA: perda 0.04658 - precisão 0.98645
